# Assignment module 3


Following the 'Actuarial Data Scientist' program offered by the Belgian association of actuaries (IABE), we are tasked with an assignment that builds on the foundational concepts learned in the third and last module.
<br/>
This assignment involves analyzing the "eusavingULnoPS" dataset which is based on unit-linked saving products, with no profit sharing, sold in an unknown European country. Those insurance policies are observed between 1999 and 2008, with entries and exits possible.
<br/>

In this analysis, we will focus on developing classification and regression models for specific targets, applying feature selection techniques, and implementing explainability and fairness assessments.
<br/>
The objective is to provide actionable insights into the factors influencing policy lapses and risk premium proportions, ensuring the models are transparent, fair, and robust.

## Importing packages <a name="Importing_packages"></a>

Our first step is to import and load the packages that will be necessary in our task.


In [4]:
## Perform these pip installs in the command prompt, with your python kernel activated
!pip install shap
!pip install tensorflow
!pip install shapicant
!pip install scikeras
!pip install mlflow
!pip install evidently
!pip install boruta
!pip install pandas scikit-learn deap
!pip install aif360

!git clone https://github.com/Regis0323/Module-3-assignment.git
!pip install scikit-fuzzy

fatal: destination path 'Module-3-assignment' already exists and is not an empty directory.


In [33]:
# 1. SETUP: load libraries & data
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Data preprocesing and models
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV
from sklearn.linear_model import LassoCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.metrics import classification_report
!pip install tensorflow


# MLFlow
import mlflow
import cloudpickle
from evidently import Dataset
from evidently import DataDefinition
from evidently import Report
from evidently.presets import DataDriftPreset, DataSummaryPreset

# Feature selection and explainability of the model
from mlxtend.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LinearRegression
from sklearn.inspection import permutation_importance
from boruta import BorutaPy
from deap import base, creator, tools, algorithms
import random

# Bias mitigation
from aif360.datasets import StandardDataset, BinaryLabelDataset
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric
from aif360.algorithms.preprocessing.optim_preproc_helpers.opt_tools import OptTools
from aif360.algorithms.preprocessing import Reweighing, OptimPreproc
from aif360.algorithms.inprocessing import MetaFairClassifier, AdversarialDebiasing
from aif360.algorithms.postprocessing import RejectOptionClassification, EqOddsPostprocessing



# Data exploration and preoprocessing


Then we load the dataset and get its specificities.

In [15]:
# Load
df = pd.read_csv("/content/Module-3-assignment/eusavingULnoPS.csv")
display(df.head())
print(df.dtypes.value_counts())

,policy.ID,issue.date,termination.date,lapse.reason,premium.frequency,gender,underwriting.age,face.amount,risk.premium,saving.premium,...,rate2Y.relvar1mth,rate2Y.relvar1qtr,rate10Y.relvar1mth,rate10Y.relvar1qtr,unemploy.relvar1mth,unemploy.relvar1qtr,industry.relvar1mth,industry.relvar1qtr,RTV.relvar1mth,RTV.relvar1qtr
0,N1,1999-01-01,2006-04-01,Claim,unique,Male,77,2979.53,265.86,29795.27,...,0.086310,0.184211,0.081598,0.144626,-0.011494,-0.011494,-0.000979,0.016699,0.005941,0.006931
1,N2,1999-01-01,2006-04-01,Claim,unique,Male,77,3039.71,271.23,30397.16,...,0.086310,0.184211,0.081598,0.144626,-0.011494,-0.011494,-0.000979,0.016699,0.005941,0.006931
2,N3,1999-01-01,2005-04-01,Surrender,unique,Female,72,2994.12,80.43,35929.44,...,-0.025391,-0.018489,-0.023343,-0.018868,-0.010309,-0.040816,0.001000,0.003024,0.002002,0.000998
3,N4,1999-01-01,2003-02-01,Surrender,unique,Male,35,3051.10,7.50,30511.01,...,-0.041762,-0.133838,-0.038309,-0.039273,0.000000,-0.008850,-0.003067,0.000000,0.009524,0.009574
4,N5,1999-01-01,2008-01-01,In force,unique,Male,40,5810.21,81.39,58102.11,...,0.037712,-0.004430,0.025938,-0.002027,0.022727,0.069767,0.001876,0.000941,-0.011550,-0.026692


float64    23
object      6
int64       1
Name: count, dtype: int64


Our dataset is composed of 30 columns, of which 6 are categorized as 'object', 1 as 64-bit 'integer' (int64) and 23 as 64-bit 'floating-point' (float64) numbers.
<br>
We shall now check for missing and/or duplicated values before treating them accordingly if any.

In [4]:
#Potential missing values
Missing_Values=df.isna().sum()
print(Missing_Values)

#Potential duplicated values
Duplicated_Values=df[df.duplicated(keep=False)]
print(Duplicated_Values)
  #Treating duplicated values (if any)
df_cleaned=df.drop_duplicates(keep='first')

#Information about the new cleaned dataset
df_cleaned.info()
print(df_cleaned.columns.tolist())

policy.ID              0
issue.date             0
termination.date       0
lapse.reason           0
premium.frequency      0
gender                 0
underwriting.age       0
face.amount            0
risk.premium           0
saving.premium         0
CPI.relvar1mth         0
CPI.relvar1qtr         0
CPI.relvar1yr          0
CPI.relvar2yr          0
EUidx.relvar1mth       0
EUidx.relvar1qtr       0
EUidx.relvar1yr        0
EUidx.relvar2yr        0
rate1Y.relvar1mth      0
rate1Y.relvar1qtr      0
rate2Y.relvar1mth      0
rate2Y.relvar1qtr      0
rate10Y.relvar1mth     0
rate10Y.relvar1qtr     0
unemploy.relvar1mth    0
unemploy.relvar1qtr    0
industry.relvar1mth    0
industry.relvar1qtr    0
RTV.relvar1mth         0
RTV.relvar1qtr         0
dtype: int64
Empty DataFrame
Columns: [policy.ID, issue.date, termination.date, lapse.reason, premium.frequency, gender, underwriting.age, face.amount, risk.premium, saving.premium, CPI.relvar1mth, CPI.relvar1qtr, CPI.relvar1yr, CPI.relvar2yr, EUidx.

After this check, we can see that there are neither missing values nor duplicates and there are 2 date variables that have been properly treated.

We shall now create two datasets (df_cat and df_num), one for each of the following two dependent variables:
*   a categorical one: lapse.reason;
*   a numeric one: a numerical variable that represents the proportion of risk premium over total premium: proportion_risk_premium = risk.premium / (risk.premium+saving.premium)

In [19]:
df_cat = df_cleaned.copy()
df_num = df_cleaned.copy()

# Creation of numerical dependent variable
df_num['proportion_risk_premium'] = 0
df_num['proportion_risk_premium'] = df_num['risk.premium']/(df_num['risk.premium'] + df_num['saving.premium'])

print(df_num.dtypes.value_counts())
print(df_cat.dtypes.value_counts())
print('Number of NaN:', df_num.isna().sum())

float64    24
object      6
int64       1
Name: count, dtype: int64
float64    23
object      6
int64       1
Name: count, dtype: int64
Number of NaN: policy.ID                  0
issue.date                 0
termination.date           0
lapse.reason               0
premium.frequency          0
gender                     0
underwriting.age           0
face.amount                0
risk.premium               0
saving.premium             0
CPI.relvar1mth             0
CPI.relvar1qtr             0
CPI.relvar1yr              0
CPI.relvar2yr              0
EUidx.relvar1mth           0
EUidx.relvar1qtr           0
EUidx.relvar1yr            0
EUidx.relvar2yr            0
rate1Y.relvar1mth          0
rate1Y.relvar1qtr          0
rate2Y.relvar1mth          0
rate2Y.relvar1qtr          0
rate10Y.relvar1mth         0
rate10Y.relvar1qtr         0
unemploy.relvar1mth        0
unemploy.relvar1qtr        0
industry.relvar1mth        0
industry.relvar1qtr        0
RTV.relvar1mth             0
RTV.relv

## Data preprocessing
After creating the numerical and categorical datasets, we shall now define the X (X_cat and X_num) and y datasets before converting categorical variables to a suitable format (one-hot encoding), handling date variables and splitting the datasets into training and test sets.

In [31]:
# Date variables handling
df_cat['termination.date'] = pd.to_datetime(df_cat['termination.date'], errors='coerce')
# Creating new numeric features from dates
df_cat['termination.date.yr'] = df_cat['termination.date'].dt.year
df_cat['termination.date.month'] = df_cat['termination.date'].dt.month
df_cat['termination.date.day'] = df_cat['termination.date'].dt.day

# Defining X and y
y_cat = df_cat['lapse.reason']
y_num = df_num['proportion_risk_premium']
X_cat = df_cat.drop(['lapse.reason'], axis=1)
X_num = df_num.drop(['proportion_risk_premium'], axis=1)


# One-hot encoding for categorical variables
categorical_cols = X_cat.select_dtypes(include=['object', 'category']).columns.tolist()
X_cat_encoded = pd.get_dummies(X_cat, columns=categorical_cols, drop_first=True)


# Split test and train datasets
  # Stratify for classification
X_train_cat, X_test_cat, y_train_cat, y_test_cat = train_test_split(X_cat, y_cat, test_size=0.2, random_state=23, stratify=y_cat)

  # No stratify for regression
X_train_num, X_test_num, y_train_num, y_test_num = train_test_split(X_num, y_num, test_size=0.2, random_state=23)

# Convert date in test set
# Note: Ensure your test set is also named appropriately, e.g., X_test_cat
X_test_cat['termination.date'] = pd.to_datetime(X_test_cat['termination.date'], errors='coerce')
X_test_cat['termination.date.yr'] = X_test_cat['termination.date'].dt.year
X_test_cat['termination.date.month'] = X_test_cat['termination.date'].dt.month
X_test_cat['termination.date.day'] = X_test_cat['termination.date'].dt.day

X_train_cat['termination.date'] = pd.to_datetime(X_train_cat['termination.date'], errors='coerce')
X_train_cat['termination.date.yr'] = X_train_cat['termination.date'].dt.year
X_train_cat['termination.date.month'] = X_train_cat['termination.date'].dt.month
X_train_cat['termination.date.day'] = X_train_cat['termination.date'].dt.day


# Drop original date columns if needed
X_test_cat.drop('termination.date', axis=1, inplace=True)
X_train_cat.drop('termination.date', axis=1, inplace=True)



In [36]:
df_cat = df_cleaned.copy()
# Convert date in the full dataset for df_cat
df_cat['termination.date'] = pd.to_datetime(df_cat['termination.date'], errors='coerce')
# Create numeric features from date
df_cat['termination.date.yr'] = df_cat['termination.date'].dt.year
df_cat['termination.date.month'] = df_cat['termination.date'].dt.month
df_cat['termination.date.day'] = df_cat['termination.date'].dt.day

# Repeat for df_num if needed
# (assuming df_num is aligned with df_cat)
# If not, apply similar conversion on the dataset used for df_num


In [37]:
# Now split, using df_cat and df_num with date features already added
X_cat = df_cat.drop(['lapse.reason'], axis=1)
y_cat = df_cat['lapse.reason']

X_num = df_num.drop(['proportion_risk_premium'], axis=1)
y_num = df_num['proportion_risk_premium']

X_train_cat, X_test_cat, y_train_cat, y_test_cat = train_test_split(
    X_cat, y_cat, test_size=0.2, random_state=23, stratify=y_cat)

X_train_num, X_test_num, y_train_num, y_test_num = train_test_split(
    X_num, y_num, test_size=0.2, random_state=23)


In [38]:
# Identify categorical columns
categorical_cols = X_train_cat.select_dtypes(include=['object', 'category']).columns.tolist()

# One-hot encode train and test
X_train_cat_encoded = pd.get_dummies(X_train_cat, columns=categorical_cols, drop_first=True)
X_test_cat_encoded = pd.get_dummies(X_test_cat, columns=categorical_cols, drop_first=True)

# Align columns to ensure same features
X_train_cat_encoded, X_test_cat_encoded = X_train_cat_encoded.align(X_test_cat_encoded, join='left', axis=1, fill_value=0)


In [40]:
# Step 1: Convert date columns in the full datasets before splitting
# Assume df_cat and df_num are your full datasets

# Identify date columns in df_cat
date_cols_full = df_cat.columns[df_cat.dtypes == 'datetime64[ns]'].tolist()

# Convert and extract date features in df_cat
for col in date_cols_full:
    df_cat[col] = pd.to_datetime(df_cat[col], errors='coerce')
    df_cat[f'{col}_yr'] = df_cat[col].dt.year
    df_cat[f'{col}_month'] = df_cat[col].dt.month
    df_cat[f'{col}_day'] = df_cat[col]
    df_cat.drop(col, axis=1, inplace=True)

# Repeat for df_num if it contains date columns (if any)
# For this example, assuming df_num has no date columns. If it does, repeat the above

# Step 2: Define X and y for both tasks
y_cat = df_cat['lapse.reason']
X_cat = df_cat.drop(['lapse.reason'], axis=1)

y_num = df_num['proportion_risk_premium']
X_num = df_num.drop(['proportion_risk_premium'], axis=1)

# Step 3: Split datasets into train/test
X_train_cat, X_test_cat, y_train_cat, y_test_cat = train_test_split(
    X_cat, y_cat, test_size=0.2, random_state=23, stratify=y_cat)

X_train_num, X_test_num, y_train_num, y_test_num = train_test_split(
    X_num, y_num, test_size=0.2, random_state=23)

# Step 4: Identify categorical columns in training set
categorical_cols = X_train_cat.select_dtypes(include=['object', 'category']).columns.tolist()

# Step 5: One-hot encode categorical features
X_train_cat_encoded = pd.get_dummies(X_train_cat, columns=categorical_cols, drop_first=True)
X_test_cat_encoded = pd.get_dummies(X_test_cat, columns=categorical_cols, drop_first=True)

# Step 6: Align columns in train and test sets
X_train_cat_encoded, X_test_cat_encoded = X_train_cat_encoded.align(
    X_test_cat_encoded, join='left', axis=1, fill_value=0)

# Now, all features are numeric and ready for modeling

# --- You can now proceed with your RandomForest and GridSearch ---

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

param_dist = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'bootstrap': [True, False]
}

rf = RandomForestClassifier(random_state=23)

random_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_dist,
    n_iter=10,
    cv=3,
    n_jobs=-1,
    verbose=2,
    scoring='accuracy',
    random_state=42
)

# Fit model
random_search.fit(X_train_cat_encoded, y_train_cat)

# Best parameters
print("Best parameters:", random_search.best_params_)

# Final model
best_rf = random_search.best_estimator_


Fitting 3 folds for each of 10 candidates, totalling 30 fits


ValueError: 
All the 30 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
30 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.12/dist-packages/sklearn/base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/sklearn/ensemble/_forest.py", line 360, in fit
    X, y = validate_data(
           ^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py", line 2961, in validate_data
    X, y = check_X_y(X, y, **check_para

In [45]:
# List of date columns in X_train_cat
date_cols_train = X_train_cat.columns[X_train_cat.dtypes == 'datetime64[ns]'].tolist()

# List of date columns in X_test_cat
date_cols_test = X_test_cat.columns[X_test_cat.dtypes == 'datetime64[ns]'].tolist()

# Convert and extract date features in training set
for col in date_cols_train:
    X_train_cat[col] = pd.to_datetime(X_train_cat[col], errors='coerce')
    X_train_cat[f'{col}_yr'] = X_train_cat[col].dt.year
    X_train_cat[f'{col}_month'] = X_train_cat[col].dt.month
    X_train_cat[f'{col}_day'] = X_train_cat[col]
    X_train_cat.drop(col, axis=1, inplace=True)

# Convert and extract date features in test set
for col in date_cols_test:
    X_test_cat[col] = pd.to_datetime(X_test_cat[col], errors='coerce')
    X_test_cat[f'{col}_yr'] = X_test_cat[col].dt.year
    X_test_cat[f'{col}_month'] = X_test_cat[col].dt.month
    X_test_cat[f'{col}_day'] = X_test_cat[col]
    X_test_cat.drop(col, axis=1, inplace=True)


In [46]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

# Define parameter grid
param_dist = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'bootstrap': [True, False]
}

rf = RandomForestClassifier(random_state=23)

random_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_dist,
    n_iter=10,
    cv=3,
    n_jobs=-1,
    verbose=2,
    scoring='accuracy',
    random_state=42
)

# Fit model
random_search.fit(X_train_cat, y_train_cat)

# Best params
print("Best parameters:", random_search.best_params_)
best_rf = random_search.best_estimator_


Fitting 3 folds for each of 10 candidates, totalling 30 fits


ValueError: 
All the 30 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.12/dist-packages/sklearn/base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/sklearn/ensemble/_forest.py", line 360, in fit
    X, y = validate_data(
           ^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py", line 2961, in validate_data
    X, y = check_X_y(X, y, **check_para

## Development of model for categorical variable: RandomForest

In this part we shall develop a Random Forest algorithm for the categorical dependent variable "lapse.reason".

In [32]:
# Define a smaller, efficient parameter distribution
param_dist = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'bootstrap': [True, False]
}

# Initialize the Random Forest classifier
rf = RandomForestClassifier(random_state=23)

# Setup RandomizedSearchCV with fewer iterations and folds
random_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_dist,
    n_iter=10,
    cv=3,
    n_jobs=-1,
    verbose=2,
    scoring='accuracy',
    random_state=42
)

# Fit the search on your training data
random_search.fit(X_train_encoded, y_train_cat)

# Output the best parameters
print("Best parameters from randomized search:", random_search.best_params_)

# Assign the best model
best_rf = random_search.best_estimator_

Fitting 3 folds for each of 10 candidates, totalling 30 fits


ValueError: 
All the 30 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
30 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.12/dist-packages/sklearn/base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/sklearn/ensemble/_forest.py", line 360, in fit
    X, y = validate_data(
           ^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py", line 2961, in validate_data
    X, y = check_X_y(X, y, **check_para

In [21]:
# --- Step 2: Make predictions on test set ---

y_pred = best_rf.predict(X_test_cat)

# --- Step 3: Evaluate the model ---

print("Classification Report:\n", classification_report(y_test_cat, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test_cat, y_pred))

# --- Step 4: Visualize feature importance ---

importances = best_rf.feature_importances_
features = X_train_cat.columns

feat_importances = pd.Series(importances, index=features).sort_values(ascending=False)

plt.figure(figsize=(12,6))
feat_importances.plot(kind='bar')
plt.title('Feature Importances from Optimized Random Forest')
plt.ylabel('Importance Score')
plt.show()

Fitting 5 folds for each of 216 candidates, totalling 1080 fits


ValueError: 
All the 1080 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
216 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.12/dist-packages/sklearn/base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/sklearn/ensemble/_forest.py", line 360, in fit
    X, y = validate_data(
           ^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py", line 2961, in validate_data
    X, y = check_X_y(X, y, **check_p

## Development of model for numerical variable (proportion): Sequential Dense NN


For the numerical variable "proportion_risk_premium", develop a Sequential Dense Neural Network.

In [13]:
scaler_num = StandardScaler()
# Normalize numerical features
X_train_num_scaled = scaler_num.fit_transform(X_train_num)
X_test_num_scaled = scaler_num.transform(X_test_num)

# Define neural network
model = models.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train_num_scaled.shape[1],)),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='linear')
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Train with early stopping
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

model.fit(
    X_train_num_scaled,
    y_train_num,
    validation_split=0.2,
    epochs=100,
    batch_size=32,
    callbacks=[early_stop],
    verbose=0
)

# Evaluate
loss, mae = model.evaluate(X_test_num_scaled, y_test_num)
print(f"Test MAE for risk proportion: {mae}")

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


138/138 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2.7012e-07 - mae: 3.2976e-04
Test MAE for risk proportion: 0.00033756138873286545


# Variable selection

## Categorical problem, feature selection


For the categorical dependent variable "lapse.reason", perform Boruta feature selection using (from boruta) BortutaPy.

In [9]:
categorical_features = list(X_train_cat.select_dtypes(include=['object']).columns)

# Create the preprocessor with sparse output
preprocessor = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ],
    remainder='passthrough'
)

# Fit and transform (sparse output)
X_train_cat_encoded = preprocessor.fit_transform(X_train_cat)

# No need to convert to dense
# Use BorutaPy directly with sparse matrix
rf_for_boruta = RandomForestClassifier(n_estimators=100, random_state=23)

# Encode y
le = LabelEncoder()
y_boruta = le.fit_transform(y_train_cat)

# Run Boruta
boruta_selector = BorutaPy(rf_for_boruta, max_iter=50, random_state=23)
boruta_selector.fit(X_train_cat_encoded, y_boruta)

# Get selected feature indices
support_mask = boruta_selector.support_

# Get feature names from the OneHotEncoder
feature_names = preprocessor.get_feature_names_out(categorical_features)
selected_features_boruta = feature_names[support_mask]
print("Boruta selected features:", list(selected_features_boruta))

TypeError: Sparse data was passed for X, but dense data is required. Use '.toarray()' to convert to a dense numpy array.

For the categorical dependent variable "lapse.reason", perform Genetic Algorith feature selection using (from deap) base, creator, tools, algorithms repspectively.

Optional: Use MLFLow to show you know how to use good development and tracking practices while training the model.

In [ ]:
# Setup MLFlow

host =
port =
mlflow.set_tracking_uri(uri=f"http://{host}:{port}")


In [ ]:
# Model packaging


In [ ]:
# Model registry


In [ ]:
# Model serving


In [ ]:
# Model monitoring


## Numerical problem, feature selection

For the numerical variable "proportion_risk_premium", perform a forward and a backward selection on explanatory variables.




In [10]:
# Standardize features
X_train_num_scaled = scaler_num.fit_transform(X_train_num)

# Forward selection
sfs_forward = SequentialFeatureSelector(
    Lasso(alpha=0.01),
    n_features_to_select=10,
    direction='forward'
)
sfs_forward.fit(X_train_num_scaled, y_train_num)
print("Forward selection features:", X_train_num.columns[sfs_forward.get_support()])

# Backward selection
sfs_backward = SequentialFeatureSelector(
    Lasso(alpha=0.01),
    n_features_to_select=10,
    direction='backward'
)
sfs_backward.fit(X_train_num_scaled, y_train_num)
print("Backward selection features:", X_train_num.columns[sfs_backward.get_support()])

NameError: name 'scaler_num' is not defined

For the numerical variable "proportion_risk_premium", apply Lasso using LassoCV.

Optional: Use MLFLow to show you know how to use good development and tracking practices while training the model.

In [ ]:
# Setup MLFlow

host =
port =
mlflow.set_tracking_uri(uri=f"http://{host}:{port}")


In [ ]:
# Model packaging


In [ ]:
# Model registry


In [ ]:
# Model serving


In [ ]:
# Model monitoring


# Explainability of the model

## Categorical problem

Explain the RandomForest model to a client, that wants to know why their quote is so high by permutation feature selection using permutation_importance.

In [ ]:
result = permutation_importance(clf_cat, X_test_cat, y_test_cat, n_repeats=10, random_state=42)
importances = pd.Series(result.importances_mean, index=X_test_cat.columns)
print("Feature importances:\n", importances.sort_values(ascending=False))

Choose another method of your preference to explain the RandomForest model.

## Numerical problem

Explain the Neural Netwok model to a client, that wants to know why their quote is so high using Shapley values.

In [ ]:
X_sample = X_test_num_scaled[:100]
explainer = shap.KernelExplainer(model.predict, X_train_num_scaled[:100])
shap_values = explainer.shap_values(X_sample)

shap.summary_plot(shap_values, X_test_num.columns)

Choose another method of your preference to explain the NN model.

# BIAS for the numerical problem

After having set a threshold = 0.0002 = 0.02%, define a variable called "risk_category" that takes values High or Low if variable "proportion_risk_premium" is higher or lower than the treshold.

Similarly, define variable "risk_category_pred" that does the same but on the basis of the values predicted by the Neural Network model. For these two variables, count the number of High and Low values for Females and Males.

In [ ]:
# Threshold = 0.0002 = 0.02%
threshold = 0.0002

df_num['risk_category'] = np.where(df_num['proportion_risk_premium'] > threshold, 'High', 'Low')
df_num['risk_category_pred'] =

Using confusion_matrix, compare real and predicted values by counting False positives, False negatives, Trues positives, True negatives.

## Metrics to detect possible bias on predicted values
Investigate possible bias on the basis of the metrics:


*   Demographic Parity (no bias range: -0.1, 0.1)
*   Equal Opportunity (no bias range: -0.1, 0.1)
*   Average Odds (no bias range: -0.1, 0.1)
*   Disparate Impact (no bias range: 0.8, 1.25)
*   Theil Index (no bias range: 0, 0.15)

## Bias mitigation
Proceed with:

*   Preprocessing bias mitigation: choose the method, between reweighing and optimized preprocessing, that leads to a greater bias reduction
*   In-processing bias mitigation:  choose the method, between Meta Fair
*   Postprocessing bias mitigation: Reject option classification and equalized Odds




In [ ]:
# Preprocessing


In [ ]:
# In-processing


In [ ]:
# Post-Processing


Compare pros and cons of chosen methods:




Write down in text what you think the pros and cons are of the methods above